In [9]:
'''The purpose of this script is to look into which rosetta F22 images view a specified 
ground point of the CG67 (within a specific degree). This cell calculates the theta angle
between the selected ground point vector and the ground point vector for the center pixel
of an input image. Theta values are stored for all input images in a dictionary of format 
{absPath/imgName: theta}.
'''

import numpy as np
import subprocess, pvl, os, json


#USER INPUTS########
#variables set for my convenience for creating my relevant perspectives
ax = 'x' #could be set to 'x' 'y' or 'z'
ori = 'pos' #could be set to positive 'pos' or negative 'neg'

#variables required for future module
gp_perspective = [1, 0, 0] #x,y,z coordinates of the ground point perspective is centered around; typically set by user
img_file = 'img_list_head.txt'


unit_gp_perspective = gp_perspective / np.linalg.norm(gp_perspective)

#read in entire input file and save as list
img_list = open(img_file).readlines()
img_dict = {} #initilize image:theta dictionary

for i in range(0,len(img_list)):
    img = img_list[i]
    cube = os.path.splitext(os.path.split(img)[-1])[0]+'.cub' #################################IS THERE a less horrid way to do this?

    #ingest image
    command = ['rososiris2isis',
               'from={}'.format(img),
               'to={}'.format(cube)]
    subprocess.run(command)
        
    #spiceinit
    command = ['spiceinit',
               'from={}'.format(cube),
               'shape=user',
               'model=$ISIS3DATA/rosetta/kernels/dsk/ROS_CG_M004_OSPGDLR_U_V1.bds',
               '-preference=/work/projects/rosetta/testing/ISIS3_mission_processing_scripts/rosetta_scripts/IsisPreferences_Bullet']
    subprocess.run(command,stdout=subprocess.DEVNULL)
        
    #campt
    command = ['campt',
               'from={}'.format(cube)]
    result = subprocess.run(command,stdout=subprocess.PIPE)
    cam_out = pvl.loads(result.stdout.decode())
        
    #calculate theta and compare to tolerance 
    gp_input = np.array(cam_out['GroundPoint']['BodyFixedCoordinate'][0]) #ground point vector of center pixel of input image
    unit_gp_input = gp_input / np.linalg.norm(gp_input) #unitize
    theta = np.arccos(np.dot(unit_gp_input, unit_gp_perspective))*180/np.pi #calc angle between perspective gp vector and images center pixel gp vector
        
    #create dictionary of filename: theta 
    img_dict.update({'{}'.format(img.strip()): theta})
    
    #clean up and print progress
    subprocess.run(['rm',
                    '{}'.format(cube)])
    subprocess.run(['echo',
                    'Image {} of {} complete'.format(i+1, len(img_list))])


KeyboardInterrupt: 

In [2]:
''''''
theta_tol = 30;

img_valid = []
for i in range(0,len(img_list)):
    #if theta, from dictionary, is within tolerance
    if img_dict['{}'.format(img_list[i].strip())] < theta_tol:
        #save img base name to list for pricinple axis perspective
        img_valid.append('{}'.format(img_list[i].strip()))

print('{} images were valid for the {} principal axis perspective with a theta tolerance of {}'.format(len(img_valid), ax+ori, theta_tol))        

#create and write a text file to list valid image names 
cur_dir = os.getcwd()
sub_dir = ax+ori
ofilename = ax+ori+'_valid_imgs.txt'
ofilepath = os.path.join(cur_dir,sub_dir,ofilename)
of = open(ofilepath,'w')
of.write('\n'.join(img_valid))
#pickle.dump(img_valid, of) #write the list of valid images to file
of.close()

#create and write a text file to use as output log image name: theta calculated
log_filename = ax+ori+'_log.txt'
log_filepath = os.path.join(cur_dir,sub_dir,log_filename)
lf = open(log_filepath,'w')
lf.write("\n".join("{}: theta = {}".format(k, v) for k, v in img_dict.items())) #write out the dictionary of img_names:theta to file
lf.close()

1 images were valid for the xpos principal axis perspective with a theta tolerance of 30
